# TTnet Model

This notebook presents the files and functions implementing the time and energy model of TTnet. All functions and parameters are contained in the `ttnet_model.py` file. Constant parameter values depend on the chose hardware (radio parameters), Glossy implementation, and TTnet implementation.

We list them in the cell below for completness.

In [1]:
# 1. Radio parameters ==
L_cal       = 3         # Bytes Length of calibration Bytes
L_header    = 5         # Bytes Length of Glossy header
Rbits       = 250       # bit/ms Radio bit rate

# 2. Glossy parameters
Tcal        = 0.096     # ms Length of calibration Bytes
Theader     = 0.16      # ms Length of LWB/Glossy header
Twu_data    = 0.13      # ms Wake-up time at slot start (radio off)
Twu_control = 1.2       # ms Radio start time for sync packets (radio off)
Tstart_slot = 0.100883333333333 # ms Radio start time (radio on)
Td          = 0.174671212121212 # ms Software delay between flood phases
TdeepWU     = 0.7417    # ms Additional setup time after from deep sleep

# 3. TTnet implementation parameters
L_beacon        = 2     # Bytes Beacon size
T_gap           = 1.5   # ms Gap time
T_gap_control   = 1.5   # ms Gap time (post_control)
T_guard         = 0.1   # ms Guard time Same for control and data slots (included in T_gap)
T_preprocess    = 2     # ms Pre-process time
T_switch        = 0.3   # ms Switch between RX/TX Includes the time for preamble and sync bytes
T_slack         = 0.25  # ms Slack time in T_slot_min() macro
T_slot_base     = 0.5   # ms Granularity of T_slot values
T_post_cb       = 1     # ms Time for the on_slot_post_cb() of the last slot
T_round_end     = 1.5   # ms Time for state-keeping at the end of the Baloo round [upper-bounded to 0.5ms]

The Glossy parameters have been measured for our specific Glossy implementation for the [DPP-cc430 platform](https://github.com/ETHZ-TEC/Baloo/wiki/platform-dpp-cc430), which is available in the [Baloo repository](https://github.com/ETHZ-TEC/Baloo), together with our [TTnet implementation](https://github.com/ETHZ-TEC/Baloo/tree/master/examples/baloo-ttnet). 

The knowledge of the Glossy parameters is necessary to properly size the various time buffers in the TTnet implementation (eg, `T_switch` and `T_preprocess`) Refer to the [Baloo Wiki](https://github.com/ETHZ-TEC/Baloo/wiki) for the definition of these TTnet parameters.

There remains four variable parameters listed in the table below:

|Parameter|Description|Value(s)|Unit|
|---|---|---|---|
|H|Provisioned diameter|4|.|
|N|Number of retransmissions|2|.|
|B|Number of slots per round|5,10,30|.|
|L|Payload size|8,16,64|Bytes|

Based on these parameters, `ttnet_model.py` provides functions that computes the equations from our TTnet model, all computed in milliseconds.

# TODO: solve mismatch

okay here I have the following problem: the numerical model matches the implementation. The description of the model in the paper not really (some things are missing in the paper). How to resove?

In [2]:
'''
Equation (1)

T_slot(L) = ceil( (H + 2*N -1) * (8*( L_header + L )/Rbits + T_switch) + T_slack )

rounded up to T_slot_base
'''


''' 
T_beacon = T_guard + (H + 2*N -1) * (8*( L_header + L_beacon )/Rbits + T_switch) + T_slack

computed in ms
'''


'''
T_round(B,L) = T_guard + T_preprocess + T_beacon + T_gap_control + B*T_slot(L) + (B-1)* T_gap + T_round_end
'''



'''
T^on_beacon = T_start_slot + T_guard + (H + 2*N -1) * (T_d + T_cal + T_header + 8*L_beacon/Rbits)
'''

'''
T^on_slot(L) = T_start_slot + T_guard + (H + 2*N -1) * (T_d + T_cal + T_header + 8*L/Rbits)
'''


'''
T^on_round = T^on_beacon + B*T^on_slot(L)
'''



'''
E = (T^on_no-round - T^on_round) / T^on_no-round
T^on_no-round(B,L) = B * (T^on_beacon + T^on_slot(L))
'''

'\nE = (T^on_no-round - T^on_round) / T^on_no-round\nT^on_no-round(B,L) = B * (T^on_beacon + T^on_slot(L))\n'

In [5]:
import plotly.io as pio
pio.templates.default = "none"

from src.ttnet_model import *
from src.ttnet_plots import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# plot_protocol_overhead()
# plot_protocol_overhead()
# plot_round_length()
plot_inheritance_results()

In [10]:
fig = plot_energy_savings()
custom_layout = {
    "title":None,
    "width":0.8*linewidth_px,
    "height":250,
    "font":{"size":14},
    "margin":dict(l=30, r=0, t=45, b=45),
    "xaxis":{'title':{'font':{'size':14}},
            },
    "yaxis":{'range':[0,55]},
    "legend":{'font':{'size':14}}
}
fig.update_layout(custom_layout)
fig.show()

# Save the figure
# plot_filename = "energy_savings.pdf"
# fig.write_image(str(plot_path/plot_filename))